##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [4]:
# Personal Finance Tracker for Google Colab

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
from google.colab import files
import io

warnings.filterwarnings('ignore')

# Initialize DataFrame to store transactions
transactions = pd.DataFrame(columns=['Date', 'Description', 'Amount', 'Category', 'Type'])

# Categories setup
income_categories = ['Salary', 'Freelance', 'Investments', 'Gifts', 'Other Income']
expense_categories = ['Food', 'Transportation', 'Housing', 'Entertainment',
                      'Healthcare', 'Education', 'Shopping', 'Utilities', 'Other Expenses']
all_categories = income_categories + expense_categories

# Category color map
category_colors = {
    'Salary': '#3498db', 'Freelance': '#9b59b6', 'Investments': '#2ecc71',
    'Gifts': '#e67e22', 'Other Income': '#1abc9c', 'Food': '#e74c3c',
    'Transportation': '#f39c12', 'Housing': '#34495e', 'Entertainment': '#d35400',
    'Healthcare': '#c0392b', 'Education': '#16a085', 'Shopping': '#8e44ad',
    'Utilities': '#7f8c8d', 'Other Expenses': '#95a5a6'
}

# Custom CSS
css = """
<style>
.finance-widget {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    background: linear-gradient(145deg, #ffffff, #f0f2f5);
    margin-bottom: 20px;
    width: 95%;
}
.finance-header {
    color: #2c3e50;
    font-size: 24px;
    font-weight: 600;
    margin-bottom: 15px;
    text-align: center;
}
.transaction-table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 15px;
}
.transaction-table th {
    background: #3498db;
    color: white;
    padding: 10px;
    text-align: left;
}
.transaction-table td {
    padding: 10px;
    border-bottom: 1px solid #ecf0f1;
}
.income-amount {
    color: #27ae60;
    font-weight: 500;
}
.expense-amount {
    color: #e74c3c;
    font-weight: 500;
}
</style>
"""
display(HTML(css))

def add_transaction(date, description, amount, category, transaction_type):
    global transactions
    new_transaction = pd.DataFrame([[date, description, amount, category, transaction_type]],
                                   columns=['Date', 'Description', 'Amount', 'Category', 'Type'])
    transactions = pd.concat([transactions, new_transaction], ignore_index=True)
    transactions['Date'] = pd.to_datetime(transactions['Date'])
    transactions.sort_values('Date', inplace=True)
    display(HTML(f'<div style="color:green;padding:10px;">Transaction added: {description} (${amount})</div>'))

def show_transactions(n=10):
    if transactions.empty:
        display(HTML('<div style="padding:20px;color:#7f8c8d;">No transactions yet</div>'))
    else:
        df = transactions.tail(n).copy()
        df['Amount'] = df.apply(lambda x: f'<span class="{"income-amount" if x["Type"]=="Income" else "expense-amount"}">'
                                              f'{"+" if x["Type"]=="Income" else "-"}${x["Amount"]:,.2f}</span>', axis=1)
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

        html = f"""
        <div class="finance-widget">
            <h3 class="finance-header">Recent Transactions</h3>
            <table class="transaction-table">
                <tr><th>Date</th><th>Description</th><th>Amount</th><th>Category</th></tr>
        """
        for _, row in df.iterrows():
            html += f"""
                <tr>
                    <td>{row['Date']}</td>
                    <td>{row['Description']}</td>
                    <td>{row['Amount']}</td>
                    <td style="color:{category_colors.get(row['Category'], '#000')}">{row['Category']}</td>
                </tr>
            """
        html += "</table></div>"
        display(HTML(html))

def show_summary():
    if transactions.empty:
        display(HTML('<div style="padding:20px;color:#7f8c8d;">No transactions yet</div>'))
        return

    income = transactions[transactions['Type'] == 'Income']['Amount'].sum()
    expenses = transactions[transactions['Type'] == 'Expense']['Amount'].sum()
    balance = income - expenses

    html = f"""
    <div class="finance-widget">
        <h3 class="finance-header">Financial Summary</h3>
        <div style="display:flex;justify-content:space-around;">
            <div style="text-align:center;">
                <div style="font-size:16px;color:#27ae60;">Income</div>
                <div style="font-size:24px;">${income:,.2f}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:16px;color:#e74c3c;">Expenses</div>
                <div style="font-size:24px;">${expenses:,.2f}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:16px;color:{"#27ae60" if balance>=0 else "#e74c3c"}">Balance</div>
                <div style="font-size:24px;">${balance:,.2f}</div>
            </div>
        </div>
    </div>
    """
    display(HTML(html))

def save_data():
    if transactions.empty:
        display(HTML('<div style="color:red;padding:10px;">No data to save!</div>'))
        return
    transactions.to_csv("transactions.csv", index=False)
    files.download("transactions.csv")

def load_data():
    global transactions
    uploaded = files.upload()
    for filename in uploaded.keys():
        try:
            transactions = pd.read_csv(io.BytesIO(uploaded[filename]))
            transactions['Date'] = pd.to_datetime(transactions['Date'])
            display(HTML(f'<div style="color:green;padding:10px;">Loaded {len(transactions)} transactions</div>'))
        except Exception as e:
            display(HTML(f'<div style="color:red;padding:10px;">Error loading file: {str(e)}</div>'))

# UI elements
date_picker = widgets.DatePicker(description='Date:', value=datetime.now())
desc_input = widgets.Text(description='Description:')
amount_input = widgets.FloatText(description='Amount:', value=0.0)
type_toggle = widgets.ToggleButtons(options=['Income', 'Expense'], description='Type:')
category_dropdown = widgets.Dropdown(options=expense_categories, description='Category:')

def update_categories(change):
    category_dropdown.options = income_categories if change['new'] == 'Income' else expense_categories
type_toggle.observe(update_categories, names='value')

add_button = widgets.Button(description='Add Transaction', button_style='success')
refresh_button = widgets.Button(description='Refresh View')
save_button = widgets.Button(description='Save Data')
load_button = widgets.Button(description='Load Data')

def on_add_click(b):
    add_transaction(
        date_picker.value,
        desc_input.value,
        amount_input.value,
        category_dropdown.value,
        type_toggle.value
    )
    desc_input.value = ''
    amount_input.value = 0.0

def on_refresh_click(b):
    clear_output(wait=True)
    display(HTML(css))
    show_main_ui()

def on_save_click(b):
    save_data()

def on_load_click(b):
    load_data()

add_button.on_click(on_add_click)
refresh_button.on_click(on_refresh_click)
save_button.on_click(on_save_click)
load_button.on_click(on_load_click)

def show_main_ui():
    display(HTML("<h1 style='color:#2c3e50;'>Personal Finance Tracker</h1>"))
    input_box = widgets.VBox([
        widgets.HBox([date_picker, type_toggle]),
        desc_input,
        widgets.HBox([amount_input, category_dropdown]),
        widgets.HBox([add_button, refresh_button, save_button, load_button])
    ])
    display(input_box)
    show_summary()
    show_transactions()

# Run the UI
show_main_ui()

Date,Description,Amount,Category
2025-05-11,teacher,"+$50,000.00",Salary
2025-05-11,lunch,"-$3,000.00",Food
2025-05-11,medicine,"-$2,000.00",Healthcare
2025-05-11,park,"-$1,000.00",Entertainment
2025-05-11,rent,"-$6,000.00",Housing


## **Introduction to Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Welcome to this Colab where you will get a quick introduction to the Python programming language and the environment used for the course's exercises: Colab.

Colab is a Python development environment that runs in the browser using Google Cloud.

For example, to print "Hello World", just hover the mouse over [ ] and press the play button to the upper left. Or press shift-enter to execute.

In [ ]:
print("Hello World")

## Functions, Conditionals, and Iteration
Let's create a Python function, and call it from a loop.

In [ ]:
def HelloWorldXY(x, y):
  if (x < 10):
    print("Hello World, x was < 10")
  elif (x < 20):
    print("Hello World, x was >= 10 but < 20")
  else:
    print("Hello World, x was >= 20")
  return x + y

for i in range(8, 25, 5):  # i=8, 13, 18, 23 (start, stop, step)
  print("--- Now running with i: {}".format(i))
  r = HelloWorldXY(i,i)
  print("Result from HelloWorld: {}".format(r))

In [ ]:
print(HelloWorldXY(1,2))

Easy, right?

If you want a loop starting at 0 to 2 (exclusive) you could do any of the following

In [ ]:
print("Iterate over the items. `range(2)` is like a list [0,1].")
for i in range(2):
  print(i)

print("Iterate over an actual list.")
for i in [0,1]:
  print(i)

print("While works")
i = 0
while i < 2:
  print(i)
  i += 1

In [ ]:
print("Python supports standard key words like continue and break")
while True:
  print("Entered while")
  break

## Numpy and lists
Python has lists built into the language.
However, we will use a library called numpy for this.
Numpy gives you lots of support functions that are useful when doing Machine Learning.

Here, you will also see an import statement. This statement makes the entire numpy package available and we can access those symbols using the abbreviated 'np' syntax.

In [ ]:
import numpy as np  # Make numpy available using np.

# Create a numpy array, and append an element
a = np.array(["Hello", "World"])
a = np.append(a, "!")
print("Current array: {}".format(a))
print("Printing each element")
for i in a:
  print(i)

print("\nPrinting each element and their index")
for i,e in enumerate(a):
  print("Index: {}, was: {}".format(i, e))

In [ ]:
print("\nShowing some basic math on arrays")
b = np.array([0,1,4,3,2])
print("Max: {}".format(np.max(b)))
print("Average: {}".format(np.average(b)))
print("Max index: {}".format(np.argmax(b)))

In [ ]:
print("\nYou can print the type of anything")
print("Type of b: {}, type of b[0]: {}".format(type(b), type(b[0])))

In [ ]:
print("\nUse numpy to create a [3,3] dimension array with random number")
c = np.random.rand(3, 3)
print(c)

In [ ]:
print("\nYou can print the dimensions of arrays")
print("Shape of a: {}".format(a.shape))
print("Shape of b: {}".format(b.shape))
print("Shape of c: {}".format(c.shape))
print("...Observe, Python uses both [0,1,2] and (0,1,2) to specify lists")

## Colab Specifics

Colab is a virtual machine you can access directly. To run commands at the VM's terminal, prefix the line with an exclamation point (!).


In [ ]:
print("\nDoing $ls on filesystem")
!ls -l
!pwd

In [ ]:
print("Install numpy")  # Just for test, numpy is actually preinstalled in all Colab instances
!pip install numpy

**Exercise**

Create a code cell underneath this text cell and add code to:


*   List the path of the current directory (pwd)
* Go to / (cd) and list the content (ls -l)

In [ ]:
!pwd
!cd /
!ls -l
print("Hello")

All usage of Colab in this course is completely free or charge. Even GPU usage is provided free of charge for some hours of usage every day.

**Using GPUs**
* Many of the exercises in the course executes more quickly by using GPU runtime: Runtime | Change runtime type | Hardware accelerator | GPU

**Some final words on Colab**
*   You execute each cell in order, you can edit & re-execute cells if you want
*   Sometimes, this could have unintended consequences. For example, if you add a dimension to an array and execute the cell multiple times, then the cells after may not work. If you encounter problem reset your environment:
  *   Runtime -> Restart runtime... Resets your Python shell
  *   Runtime -> Restart all runtimes... Will reset the Colab image, and get you back to a 100% clean environment
* You can also clear the output in the Colab by doing: Edit -> Clear all outputs
* Colabs in this course are loaded from GitHub. Save to your Google Drive if you want a copy with your code/output: File -> Save a copy in Drive...

**Learn More**
*   Check out [this](https://www.youtube.com/watch?v=inN8seMm7UI&list=PLQY2H8rRoyvwLbzbnKJ59NkZvQAW9wLbx&index=3) episode of #CodingTensorFlow, and don't forget to subscribe to the YouTube channel ;)
